In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

#Load Data
data = pd.read_csv("CHI_2019_FULL.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)

#Load data about speakers
char_data = pd.read_csv("CHI_2019_CharacterData.csv")  

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Save new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

#Now, we only want to have data containing information about the answers
#For that we need to extract from the filename column, whether the file was part of an answer
#a = answer, p = presentation, q = question
#sentence_type should be the same for all tables, but just to be sure
arval_sentence_type = df_ar_val_char.Filename.str.replace('\d+','').str[3:-4]
df_ar_val_char['SentenceType'] = arval_sentence_type
emo_sentence_type = df_emotion_char.Filename.str.replace('\d+','').str[3:-4]
df_emotion_char['SentenceType'] = emo_sentence_type
aff_sentence_type = df_affect_char.Filename.str.replace('\d+','').str[3:-4]
df_affect_char['SentenceType'] = aff_sentence_type
loi_sentence_type = df_loi_char.Filename.str.replace('\d+','').str[3:-4]
df_loi_char['SentenceType'] = loi_sentence_type

#Now select only those who have SentenceType == 'a'
df_ar_val_char = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'q']
df_emotion_char = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'q']
df_affect_char = df_affect_char.loc[df_affect_char['SentenceType'] == 'q']
df_loi_char = df_loi_char.loc[df_loi_char['SentenceType'] == 'q']

## Let's start with the character feature 'sex'

In [2]:
# Start with model and sex
print('Emotion')
df_emo_X = df_emotion_char[emotion_label]
df_emo_X = sm.add_constant(df_emo_X)
df_emo_sex_Y = df_emotion_char['Sex']
df_emo_sex_Y.replace({'Male': 0.0, 'Female':1.0}, inplace = True)
logreg_emo_sex = sm.Logit(df_emo_sex_Y, df_emo_X).fit()
print(logreg_emo_sex.summary())

print('\nAffect')
df_aff_X = df_affect_char[affect_label]
df_aff_X = sm.add_constant(df_aff_X)
df_aff_sex_Y = df_affect_char['Sex']
df_aff_sex_Y.replace({'Male': 0.0, 'Female':1.0}, inplace = True)
logreg_aff_sex = sm.Logit(df_aff_sex_Y, df_aff_X).fit()
print(logreg_aff_sex.summary())

print('Level of Interest')
df_loi_X = df_loi_char[loi_label]
df_loi_X = sm.add_constant(df_loi_X)
df_loi_sex_Y = df_loi_char['Sex']
df_loi_sex_Y.replace({'Male': 0.0, 'Female':1.0}, inplace = True)
logreg_loi_sex = sm.Logit(df_loi_sex_Y, df_loi_X).fit()
print(logreg_loi_sex.summary())

print('Arousal-Valence')
df_arval_X = df_ar_val_char[['Arousal', 'Valence']]
df_arval_X = sm.add_constant(df_arval_X)
df_arval_sex_Y = df_ar_val_char['Sex']
df_arval_sex_Y.replace({'Male': 0.0, 'Female':1.0}, inplace = True)
logreg_ar_val_sex = sm.Logit(df_arval_sex_Y, df_arval_X).fit()
print(logreg_ar_val_sex.summary())

Emotion
         Current function value: 0.529345
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                    Sex   No. Observations:                  199
Model:                          Logit   Df Residuals:                      191
Method:                           MLE   Df Model:                            7
Date:                Mon, 02 Nov 2020   Pseudo R-squ.:                  0.1946
Time:                        13:32:54   Log-Likelihood:                -105.34
converged:                      False   LL-Null:                       -130.79
                                        LLR p-value:                 9.587e-09
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1409.0720   2.15e+05      0.007      0.995    -4.2e+05    4.23e+05
Anger       -1481.2972   2.15e+05     -0.007      0.99

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


### Native Speaker

In [3]:
df_emotion_nat = df_emotion_char.drop(['Char_ID', 'ID', 'Filename', 'Name','Academic Status', 'VideoID','VideoTitle', 'Sex'], axis = 1)
df_emotion_nat.dropna(inplace = True)
df_affect_nat = df_affect_char.drop(['Char_ID', 'ID', 'Filename', 'Name','Academic Status', 'VideoID','VideoTitle', 'Sex'], axis = 1)
df_affect_nat.dropna(inplace = True)
df_loi_nat = df_loi_char.drop(['Char_ID', 'ID', 'Filename', 'Name', 'VideoID','Academic Status','VideoTitle', 'Sex'], axis = 1)
df_loi_nat.dropna(inplace = True)
df_arval_nat = df_ar_val_char.drop(['Char_ID', 'ID', 'Filename', 'Name', 'Academic Status','VideoID','VideoTitle', 'Sex'], axis = 1)
df_arval_nat.dropna(inplace = True)

# Start with model and sex
print('Emotion')
df_emo_nat_X = df_emotion_nat[emotion_label]
df_emo_nat_X = sm.add_constant(df_emo_nat_X)
df_emo_nat_Y = df_emotion_nat['IsNativeSpeaker']
logreg_emo_nat = sm.MNLogit(df_emo_nat_Y, df_emo_nat_X).fit()
print(logreg_emo_nat.summary())

print('\nAffect')
df_aff_nat_X = df_affect_nat[affect_label]
df_aff_nat_X = sm.add_constant(df_aff_nat_X)
df_aff_nat_Y = df_affect_nat['IsNativeSpeaker']
logreg_aff_nat = sm.MNLogit(df_aff_nat_Y, df_aff_nat_X).fit()
print(logreg_aff_nat.summary())

print('Level of Interest')
df_loi_nat_X = df_loi_nat[loi_label]
df_loi_nat_X = sm.add_constant(df_loi_nat_X)
df_loi_nat_Y = df_loi_nat['IsNativeSpeaker']
logreg_loi_nat = sm.MNLogit(df_loi_nat_Y, df_loi_nat_X).fit()
print(logreg_loi_nat.summary())

print('Arousal-Valence')
df_arval_nat_X = df_arval_nat[['Arousal', 'Valence']]
df_arval_nat_X = sm.add_constant(df_arval_nat_X)
df_arval_nat_Y = df_arval_nat['IsNativeSpeaker']
logreg_ar_val_aca = sm.MNLogit(df_arval_nat_Y, df_arval_nat_X).fit()
print(logreg_ar_val_aca.summary())

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Emotion
         Current function value: 0.849694
         Iterations: 35
                          MNLogit Regression Results                          
Dep. Variable:        IsNativeSpeaker   No. Observations:                  199
Model:                        MNLogit   Df Residuals:                      183
Method:                           MLE   Df Model:                           14
Date:                Mon, 02 Nov 2020   Pseudo R-squ.:                 0.05435
Time:                        13:32:55   Log-Likelihood:                -169.09
converged:                      False   LL-Null:                       -178.81
                                        LLR p-value:                    0.1489
IsNativeSpeaker=Europ. Non-Native       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                               550.1919   3.18e+05      0.002      0.999   -6.22e+05 

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
